## 1. Declarations

In [1]:
# pip install setuptools==58
# pip install feedparser --upgrade
# pip install pygooglenews
# pip install vaderSentiment
# !pip install newspaper3k --quiet
import os
import numpy as np

In [2]:
import requests
import re
import urllib.request
import json
import pandas as pd
import datetime
from GoogleNews import GoogleNews
from bs4 import BeautifulSoup
from requests_html import HTMLSession
# from pygooglenews import GoogleNews
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from newspaper import Article
from newspaper import Config
import nltk
nltk.download('punkt')
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ioncararus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2. Functions and parameters declarations

In [3]:
xpath_pattern = "//link[@rel='canonical']/@href" # pattern to extract canonical url from html doc
regex_pattern = r".?([\w\-\.]+)" # regex pattern to get the domain of the website

In [4]:
def load_all_to_df(*datasets):
    """
    Input : strings of CSVs titles
    Output: pandas dataframe containg all records in all CSVs
    """
    df_list = [pd.read_csv(dataset) for dataset in datasets]
    return pd.concat(df_list, ignore_index=True)

In [5]:
def run_sent_analysis(text):
    """
    Input: text
    Output: value compound sentiment
    """
    sa = SentimentIntensityAnalyzer()
    dict_sa = sa.polarity_scores(text)
    return dict_sa['compound']

In [6]:
def get_links(filepath,filename):        
    with open(filepath+filename,"r") as f:
        return [line for count, line in enumerate(f) if 'support.google' not in line]

In [7]:
def get_article_sentiment(links_list):
    """
    Input:
        list of URLs
    Description:
        reads URL article and retrieves keywords, summary, text, and calculates sentiment of the summary
    Output:
        pandas DataFrame
    """
    
    data = {'url':[],'keywords':[],'summary':[],'text':[],'sentiment':[]}
    config = Config()
    config.browser_user_agent = USER_AGENT
#     config.request_timeout = 10
    for link in links_list:
        try:
            article = Article(link, config=config)
            article.download()
            article.parse()

            article.nlp()
            text = article.text
            summary = article.summary
            keywords = article.keywords
            keywords.sort()

            article = None
            
            _ = re.findall(regex_pattern, link)[1]
            data['url'].append(_)
            data['keywords'].append(keywords)
            data['summary'].append(summary)
            data['text'].append(text)
            data['sentiment'].append(run_sent_analysis(summary))
            
        except:
            pass
        
    df = pd.DataFrame.from_dict(data)
        
    return df

In [29]:
def return_analyzed_df(dfa, n):
    df = None
    for i in range(n):
        if i==0:
            links_list = get_links(r"/Users/ioncararus/Downloads/politicians/", f"pol_{i}.txt")
            df = get_article_sentiment(links_list)
            df['name'] = dfa.loc[i,'name']
        else:
            links_list = get_links(r"/Users/ioncararus/Downloads/politicians/", f"pol_{i}.txt")
            _df = get_article_sentiment(links_list)
            _df['name']=dfa.loc[i,'name']
            df = pd.concat([df,_df], ignore_index=True)
        if i%50==0:
            print(i, end = '\n')
        else:
            print(i, end = ' ')
    df.to_csv('sentiment.csv', sep=';')
    return df

### Preliminary Data Analysis

In [9]:
df_members = load_all_to_df("term-116_congress.csv","term-116_senate.csv")

In [37]:
df_members_sentiment = return_analyzed_df(df_members, df_members.shape[0])